In [8]:
import sys
sys.path.append("/mnt/code")

In [9]:
import argparse
import os
import mlflow
import tensorflow
from filelock import FileLock
from tensorflow.keras.datasets import mnist

import ray
from ray import train, tune
from ray.tune.schedulers import AsyncHyperBandScheduler
from ray.air.integrations.keras import ReportCheckpointCallback
#from domino_mlflow_utils.keras import ReportCheckpointCallback2
from domino_mlflow_utils.mlflow_utilities import DominoMLflowUtilities

In [10]:
service_host = os.environ["RAY_HEAD_SERVICE_HOST"]
service_port = os.environ["RAY_HEAD_SERVICE_PORT"]
print(ray.is_initialized())

if not ray.is_initialized():

    address=f"ray://{service_host}:{service_port}"
    temp_dir='/mnt/data/{}/'.format(os.environ['DOMINO_PROJECT_NAME']) #set to a dataset
    print(temp_dir)
    ray.init(address=address, _temp_dir=temp_dir, runtime_env={"py_modules": ['/mnt/code/domino_mlflow_utils']})

print('Ray Initializied')
print(f'Ray Host={service_host} and Ray Port={service_port}')

False
/mnt/data/MLFLOW_WANDB/


2024-01-11 00:26:44,483	INFO packaging.py:530 -- Creating a file package for local directory '/mnt/code/domino_mlflow_utils'.
2024-01-11 00:26:44,486	INFO packaging.py:358 -- Pushing file package 'gcs://_ray_pkg_4c4e9e80ccc29da2.zip' (0.07MiB) to Ray cluster...
2024-01-11 00:26:44,489	INFO packaging.py:371 -- Successfully pushed file package 'gcs://_ray_pkg_4c4e9e80ccc29da2.zip'.
SIGTERM handler is not set because current thread is not the main thread.


Ray Initializied
Ray Host=ray-659f065bc6aa262f4114e8b6-ray-client.domino-compute.svc.cluster.local and Ray Port=10001


In [11]:
experiment_name = 'FRAC-GPU-TEST'+'-' + os.environ['DOMINO_STARTING_USERNAME'] + '-' + os.environ['DOMINO_PROJECT_NAME']
mlflow_tracking_uri = os.environ['CLUSTER_MLFLOW_TRACKING_URI']
client = mlflow.tracking.MlflowClient()
experiment = client.get_experiment_by_name(name=experiment_name)
if(experiment is None):
    print('Creating experiment ')
    client.create_experiment(name=experiment_name)
    experiment = client.get_experiment_by_name(name=experiment_name)

print(experiment_name)
mlflow.set_experiment(experiment_name=experiment_name)


FRAC-GPU-TEST-sameer_wadkar-MLFLOW_WANDB


<Experiment: artifact_location='mlflow-artifacts:/mlflow', creation_time=1704929421852, experiment_id='452', last_update_time=1704929421852, lifecycle_stage='active', name='FRAC-GPU-TEST-sameer_wadkar-MLFLOW_WANDB', tags={'mlflow.domino.dataset_info': '65660f8009b68e2720e6c80c-65660f8009b68e2720e6c80b',
 'mlflow.domino.environment_id': '659ee75412b6f32d73f252e0',
 'mlflow.domino.environment_revision_id': '659f02e8c6aa262f4114e8af',
 'mlflow.domino.hardware_tier': 'small-k8s',
 'mlflow.domino.project_id': '65660f7f09b68e2720e6c808',
 'mlflow.domino.project_name': 'MLFLOW_WANDB',
 'mlflow.domino.run_id': '659f065bc6aa262f4114e8b6',
 'mlflow.domino.run_number': '14',
 'mlflow.domino.user': 'sameer_wadkar',
 'mlflow.domino.user_id': '6283a3966d4fd0362f8ba2a8',
 'mlflow.source.type': 'NOTEBOOK',
 'mlflow.user': 'sameer_wadkar'}>

In [12]:
from ray.air._internal.mlflow import _MLflowLoggerUtil
mlflow_util = _MLflowLoggerUtil()
def initialize_run():    
    experiment_name = 'FRAC-GPU-TEST'+'-' + os.environ['DOMINO_STARTING_USERNAME'] + '-' + os.environ['DOMINO_PROJECT_NAME']
    client = mlflow.tracking.MlflowClient()
    mlflow_tracking_uri = os.environ['CLUSTER_MLFLOW_TRACKING_URI']

    mlflow_util.setup_mlflow(
            tracking_uri=mlflow_tracking_uri,            
            experiment_name=experiment_name,
        )
    now = round(time.time())
    now_str=time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime(now))



    mlflow_util.start_run(tags={}, run_name=f"root-{now_str}")
    return run.info.run_id

In [13]:


import tensorflow as tf
temp_dir='/mnt/data/{}/'.format(os.environ['DOMINO_PROJECT_NAME']) #set to a dataset
experiment_name = 'FRAC-GPU-TEST'+'-' + os.environ['DOMINO_STARTING_USERNAME'] + '-' + os.environ['DOMINO_PROJECT_NAME']
client = mlflow.tracking.MlflowClient()
mlflow_tracking_uri = os.environ['CLUSTER_MLFLOW_TRACKING_URI']

def train_mnist(config):
    # https://github.com/tensorflow/tensorflow/issues/32159
    

    batch_size = 128
    num_classes = 10
    epochs = 12
    parent_run_id = config['parent_run_id']

    with FileLock(os.path.expanduser("~/.data.lock")):
        (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train, x_test = x_train / 255.0, x_test / 255.0
    model = tf.keras.models.Sequential(
        [
            tf.keras.layers.Flatten(input_shape=(28, 28)),
            tf.keras.layers.Dense(config["hidden"], activation="relu"),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Dense(num_classes, activation="softmax"),
        ]
    )

    model.compile(
        loss="sparse_categorical_crossentropy",
        optimizer=tf.keras.optimizers.SGD(lr=config["lr"], momentum=config["momentum"]),
        metrics=["accuracy"],
    )
    #cb = ReportCheckpointCallback2(metrics={"mean_accuracy": "accuracy"})
    
    mlflow.tensorflow.autolog()
    run_tags={}
    run_tags["mlflow.parentRunId"] = parent_run_id
    mlflow_utils = DominoMLflowUtilities()    
    mlflow_utils.init(experiment_name,config,run_tags=run_tags)

    model.fit(
            x_train,
            y_train,
            batch_size=batch_size,
            epochs=epochs,
            verbose=0,
            validation_data=(x_test, y_test),
            callbacks=[ReportCheckpointCallback(metrics={"mean_accuracy": "accuracy"})],
        )
    mlflow_utils.finish()


def tune_mnist(parent_run_id):
    sched = AsyncHyperBandScheduler(
        time_attr="training_iteration", max_t=400, grace_period=20
    )

    tuner = tune.Tuner(
        tune.with_resources(train_mnist, resources={"cpu": 1, "gpu": 0}),
        tune_config=tune.TuneConfig(
            metric="mean_accuracy",
            mode="max",
            scheduler=sched,
            num_samples=10,            
        ),
        run_config=train.RunConfig(
            name="exp",
            stop={"mean_accuracy": 0.99},
            storage_path=temp_dir,
        ),
        param_space={
            "threads": 2,
            "lr": tune.uniform(0.001, 0.1),
            "momentum": tune.uniform(0.1, 0.9),
            "hidden": tune.randint(32, 512),
            "parent_run_id": parent_run_id
        },
    )
    results = tuner.fit()
    
    print("Best hyperparameters found were: ", results.get_best_result().config)
   

In [14]:
with mlflow.start_run() as run:
    parent_run_id = run.info.run_id
    print(parent_run_id)
    tune_mnist(parent_run_id)


(TunerInternal pid=15510) [output] This will use the new output engine with verbosity 1. To disable the new output and use the legacy output engine, set the environment variable RAY_AIR_NEW_OUTPUT=0. For more information, please see https://github.com/ray-project/ray/issues/36949


(TunerInternal pid=15510) ╭──────────────────────────────────────────────────────────╮
(TunerInternal pid=15510) │ Configuration for experiment     exp                     │
(TunerInternal pid=15510) ├──────────────────────────────────────────────────────────┤
(TunerInternal pid=15510) │ Search algorithm                 BasicVariantGenerator   │
(TunerInternal pid=15510) │ Scheduler                        AsyncHyperBandScheduler │
(TunerInternal pid=15510) │ Number of trials                 10                      │
(TunerInternal pid=15510) ╰──────────────────────────────────────────────────────────╯
(TunerInternal pid=15510) 
(TunerInternal pid=15510) View detailed results here: /mnt/data/MLFLOW_WANDB/exp
(TunerInternal pid=15510) To visualize your results with TensorBoard, run: `tensorboard --logdir /home/ray/ray_results/exp`
(TunerInternal pid=15510) 
(TunerInternal pid=15510) Trial status: 10 PENDING
(TunerInternal pid=15510) Current time: 2024-01-10 16:27:00. Total running time: 

(TunerInternal pid=15510) AIR_VERBOSITY is set, ignoring passed-in ProgressReporter for now.
(pid=2689, ip=10.0.114.178) 2024-01-10 16:27:03.542076: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
(pid=3516, ip=10.0.63.23) 2024-01-10 16:27:04.042724: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
(pid=3214, ip=10.0.108.213) 2024-01-10 16:27:04.151946: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
(pid=3244, ip=10.0.108.154) 2024-01-10 16:27:04.130871: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
(pid=3415, ip=10.0.100.59) 2024-01-10 16:27:04.271023: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


(TunerInternal pid=15510) 
(TunerInternal pid=15510) Trial train_mnist_20682_00000 started with configuration:
(TunerInternal pid=15510) ╭───────────────────────────────────────────────────────────────╮
(TunerInternal pid=15510) │ Trial train_mnist_20682_00000 config                          │
(TunerInternal pid=15510) ├───────────────────────────────────────────────────────────────┤
(TunerInternal pid=15510) │ hidden                                                    457 │
(TunerInternal pid=15510) │ lr                                                    0.04496 │
(TunerInternal pid=15510) │ momentum                                              0.16427 │
(TunerInternal pid=15510) │ parent_run_id                            ...4a2bb4feb4d87b362 │
(TunerInternal pid=15510) │ threads                                                     2 │
(TunerInternal pid=15510) ╰───────────────────────────────────────────────────────────────╯


(train_mnist pid=2689, ip=10.0.114.178) 2024-01-10 16:27:05.942997: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
(train_mnist pid=2689, ip=10.0.114.178) 2024-01-10 16:27:05.943051: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
(train_mnist pid=2689, ip=10.0.114.178) 2024-01-10 16:27:05.943067: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ray-659f065bc6aa262f4114e8b6-ray-worker-2): /proc/driver/nvidia/version does not exist
(train_mnist pid=2689, ip=10.0.114.178) 2024-01-10 16:27:05.943268: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instruct

...

(TunerInternal pid=15510) 
(TunerInternal pid=15510) Trial train_mnist_20682_00009 errored after 0 iterations at 2024-01-10 16:27:15. Total running time: 14s
(TunerInternal pid=15510) Error file: /home/ray/ray_results/exp/train_mnist_20682_00009_9_hidden=343,lr=0.0659,momentum=0.3241_2024-01-10_16-27-00/error.txt


2024-01-11 00:27:17,709	WARNING experiment_analysis.py:584 -- Could not find best trial. Did you pass the correct `metric` parameter?


(TunerInternal pid=15510) 
(TunerInternal pid=15510) Trial status: 10 ERROR
(TunerInternal pid=15510) Current time: 2024-01-10 16:27:17. Total running time: 17s
(TunerInternal pid=15510) Logical resource usage: 0/6 CPUs, 0/0 GPUs
(TunerInternal pid=15510) ╭────────────────────────────────────────────────────────────────────────╮
(TunerInternal pid=15510) │ Trial name                status            lr     momentum     hidden │
(TunerInternal pid=15510) ├────────────────────────────────────────────────────────────────────────┤
(TunerInternal pid=15510) │ train_mnist_20682_00000   ERROR      0.044958      0.164275        457 │
(TunerInternal pid=15510) │ train_mnist_20682_00001   ERROR      0.0531664     0.551038        142 │
(TunerInternal pid=15510) │ train_mnist_20682_00002   ERROR      0.0483602     0.860322        168 │
(TunerInternal pid=15510) │ train_mnist_20682_00003   ERROR      0.0819864     0.769916        119 │
(TunerInternal pid=15510) │ train_mnist_20682_00004   ERROR    

(TunerInternal pid=15510) Experiment checkpoint syncing has been triggered multiple times in the last 30.0 seconds. A sync will be triggered whenever a trial has checkpointed more than `num_to_keep` times since last sync or if 300 seconds have passed since last sync. If you have set `num_to_keep` in your `CheckpointConfig`, consider increasing the checkpoint frequency or keeping more checkpoints. You can supress this warning by changing the `TUNE_WARN_EXCESSIVE_EXPERIMENT_CHECKPOINT_SYNC_THRESHOLD_S` environment variable.
(TunerInternal pid=15510) Trials did not complete: [train_mnist_20682_00000, train_mnist_20682_00001, train_mnist_20682_00002, train_mnist_20682_00003, train_mnist_20682_00004, train_mnist_20682_00005, train_mnist_20682_00006, train_mnist_20682_00007, train_mnist_20682_00008, train_mnist_20682_00009]


RuntimeError: No best trial found for the given metric: mean_accuracy. This means that no trial has reported this metric, or all values reported for this metric are NaN. To not ignore NaN values, you can set the `filter_nan_and_inf` arg to False.

In [ ]:
#!pip install tblib